# Base de classification en Turc

    - Transformers : Summarization : 2 modèles --> 2 Résumés / Puis score de similarités de ces 2 résumés
    Noter que l'on peut faire aussi la similarité des textes (autre note ?) et non du résumé
    - Text classification sur une base de catégories "Presse" : sport - actaulités - économie - etc
    - Sentiment analysis : voir si le ton du texte est de même type 
    - Les 2 derniers classifier seronts utilisés en produit scalaire : Par Catégorie : texte1: note1 - texte2 : note2
    et donc sum(notes_par_catégorie) = sum(note1*note2) * 100 au bout (note sur 100)

In [1]:
import pandas as pd
import pke
import spacy
import torch
import stanza
import spacy_stanza
import warnings
import string
from gensim.models import KeyedVectors
import enchant    # Pour correction orthographique de synonymes
import numpy as np
import re
from transformers import pipeline
from transformers import AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import AutoModelWithLMHead, AutoTokenizer
from transformers import BertTokenizerFast, EncoderDecoderModel
from transformers import AutoModelForTokenClassification, AutoTokenizer
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
tqdm.pandas()
warnings.filterwarnings("ignore")

C:\Users\stg-sdu\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\stg-sdu\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\stg-sdu\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


**Selection des modèles NLP : ici TURC**

In [3]:
dico_spacy = {}   # 'en':nlp_en,'de':nlp_de,'es':nlp_es,'pl':nlp_pl  - POUR MEMOIRE
langues = ['en','fr','es','de','pl','ar','tr']

In [4]:
# Chargement du modèle Word2Vec pour utilisation de synonymes
model_gensim = KeyedVectors.load_word2vec_format("D:/Users/STG-SDU/Documents/NLP/turkish", binary=True, unicode_errors="ignore")

In [5]:
# Stopwords Français NLTK + Spacy 
#stopWords = list(nlp_fr.Defaults.stop_words)
stopwords_tr = list(stopwords.words('turkish'))  
#stopwords_fr = list(set(stopwords_fr + stopWords))
stopwds_lg = {'tr':stopwords_tr}

In [6]:
# Named entity recognition turc 
model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner=pipeline('ner', model=model, tokenizer=tokenizer)

**Sélection des modèles Transformers : Summary - Text Classification - Sentiment Analysis - Similarity**

In [7]:
# Modèle transformers résumé 1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt = 'mrm8488/bert2bert_shared-turkish-summarization'
tokenizer = BertTokenizerFast.from_pretrained(ckpt)
model = EncoderDecoderModel.from_pretrained(ckpt).to(device)

def summarizer1(text):
    inputs = tokenizer([text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    output = model.generate(input_ids, attention_mask=attention_mask)
    return tokenizer.decode(output[0], skip_special_tokens=True)

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [8]:
# Modèles Transformers de Résumé (NB : Ne pas oublier d'ajouter la truncation pour tous les modèles, peut être source d'erreur)
#summarizer1 = pipeline("summarization", model="mrm8488/bert2bert_shared-turkish-summarization", truncation = "only_first")
summarizer2 = pipeline("summarization", model="ozcangundes/mt5-small-turkish-summarization", truncation = "only_first")

In [9]:
# Textes classification ou Zero shot classification (permet de chosiri nos propres thèmes)
text_clf1 = pipeline("text-classification", model = "savasy/bert-turkish-text-classification", truncation = "only_first")   # 10 actégories, voir hugging face
# labels = ['world','economy','technology','health','politics','sport','culture']

text_clf2 = pipeline('zero-shot-classification', model="serdarakyol/interpress-turkish-news-classification",truncation = "only_first")
candidate_labels = ['Culture-Art','Politics','Economy','World','Health','Technology','Agenda', 'Sport','Education','Magazine']

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [10]:
# Sentiment Analysis
sentiment1 = pipeline("text-classification", model = 'savasy/bert-base-turkish-sentiment-cased')
sentiment2 = pipeline('sentiment-analysis', model="symanto/xlm-roberta-base-snli-mnli-anli-xnli")

In [11]:
# ENCODAGE AVEC SENTENCE TRANSFORMER
from sentence_transformers import SentenceTransformer,util
encoder = SentenceTransformer("symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli")
def score_similarite(sentence1,sentence2):
    # attention, pour que torch fonctionne en dimension sentence1 (et 2) est une liste simple
    embed1 = encoder.encode(sentence1, convert_to_tensor=True)
    embed2 = encoder.encode(sentence2, convert_to_tensor=True)
    return round(float(util.pytorch_cos_sim(embed1,embed2))*100,2)

**Selection Data par langues**

In [218]:
data = pd.read_csv('eval_data_prep_v1.csv')

In [219]:
data

,pair_id,pair_lang,source_url_1,publish_date_1,source_url_2,publish_date_2,title_1,text_1,meta_description_1,meta_keywords_1,title_2,text_2,meta_description_2,meta_keywords_2,ligne
0,1484189203_1484121193,en_en,https://wsvn.com,NaN,https://wsvn.com,NaN,Police: 2 men stole tools from Lowe’s in Davie,"DAVIE, FLA. (WSVN) - Police need help catching...",NaN,[''],No-swim advisory lifted for Deerfield Beach Pier,"DEERFIELD BEACH, FLA. (WSVN) - A no-swim advis...",NaN,[''],0
1,1484011097_1484011106,en_en,https://www.zdnet.com,NaN,https://securityboulevard.com,Fri Oct 25 11:10:18 2019,"Open database leaked 179GB in customer, US gov...",Govt officials confirm Trump can block US comp...,The US Department of Homeland Security has bec...,[''],Best Western’s Massive Data Leak: 179GB Amazon...,The latest huge unsecured cloud storage find i...,The latest huge unsecured cloud storage find i...,[''],1
2,1484039488_1484261803,en_en,https://www.presstelegram.com,Tue Dec 31 00:00:00 2019,https://boingboing.net,Wed Jan 1 00:00:00 2020,Ducks are own worst enemies in sloppy loss in ...,"Ducks defenseman Erik Gudbranson, left, knocks...",NaN,[''],Woody Guthrie's 1943 New Year's Resolutions ar...,Woody Guthrie's 1943 New Year's Resolutions ar...,"I'd seen this before, but I was reminded of it...",[''],2
3,1484332324_1484796748,en_en,https://www.financialexpress.com,Thu Jan 2 08:28:22 2020,https://www.news18.com,NaN,Another Bengal vs Centre tussle? Govt rejects ...,The West Bengal government’s proposal was reje...,The West Bengal government's proposal was reje...,"['republic day', 'west bengal tableau', 'benga...",'Congress Rejected 7 Times': BJP's Reminder as...,Mumbai: The NCP and Shiv Sena on Thursday targ...,BJP ally and Union minister Ramdas Athawale sa...,"['BJP', 'congress', 'Mamata Banerjee', 'NCP', ...",3
4,1484012256_1484419682,en_en,https://www.birminghammail.co.uk,Wed Jan 1 15:03:04 2020,http://m.fightbacknews.org,Wed Jan 1 00:00:00 2020,Bars and clubs you loved and lost this decade ...,The video will start in 8 Cancel\n\nSign up to...,Nightclubs and bars that have closed in the pa...,"['Birmingham City Centre', 'Digbeth', 'Things ...",Top 20 films of the 2010s,"Jacksonville, FL - I'm not sure how we'll look...","Jacksonville, FL - I'm not sure how we'll look...","['organizing', 'activism', 'socialism', ""Peopl...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4948,1553907621_1553488848,es_it,https://www.diariolibre.com,NaN,https://www.basketuniverso.it,Thu Mar 19 23:45:49 2020,Denver Nuggets reporta que “un miembro de la o...,Los Denver Nuggets de la NBA reportaron este j...,Los Denver Nuggets de la NBA reportaron este j...,['NBA'],"Coronavirus, un caso anche fra i Denver Nuggets",Nato ad Alatri (Fr) nel ’93 e qui diplomato al...,"Un altro caso di Coronavirus nella NBA, stavol...",[''],4948
4949,1646957948_1643667075,es_it,https://diario16.com,Sun Jun 28 04:20:00 2020,https://www.laregione.ch,Wed Jun 24 16:41:00 2020,Vivir en España es más barato que en la media ...,El estudio realizado por Eurostat muestra que ...,El estudio realizado por Eurostat muestra que ...,[''],"Coronavirus, in Europa 140mila morti in più in...","Nei mesi di marzo e aprile 2020, dalla decima ...",Il picco di morti aggiuntivi rispetto alla med...,[''],4949
4950,1504063453_1502866628,es_it,https://elaragueno.com.ve,NaN,https://www.greenme.it,Thu Jan 23 12:46:02 2020,Activan sistema de vigilancia epidemiológica e...,Foto: Archivo Foto: Archivo\n\nEl sistema de v...,NaN,[''],Coronavirus in Cina: consumo di serpenti e zup...,La diffusione del mortale Coronavirus potrebbe...,L'infezione da coronavirus potrebbe aver avuto...,"['cina', 'coronavirus', 'pipistrelli', 'serpen...",4950
4951,1647862428_1647712939,es_it,http://www.am.com.mx,Mon Jun 29 00:00:00 2020,https://it.sputniknews.com,Mon Jun 29 13:46:00 2020,Emite Irán orden de arresto contra Tump por as...,CDMX.- Anunció Irán este lunes que ha emitido ...,"Ha emitido Irán una orden de arresto, que ha

In [220]:
# remémorer numéro de ligne - compléter les Nan
data['ligne'] = data.index
data = data.fillna('')

In [221]:
# séparation des datasets, le dernier étant à traduire en plus
turc = data.loc[data.pair_lang == 'tr_tr',['ligne','title_1','title_2','text_1','text_2']].reset_index(drop=True)

In [222]:
turc.iloc[0:50,:]

,ligne,title_1,title_2,text_1,text_2
0,76,ŞOK 25-31 Mart Aktüel ürünler kataloğu 2020! Ş...,Endüstriyel metallerde düşüş bakırın liderliği...,Sizlere daha iyi bir hizmet sunabilmek için si...,Endüstriyel metallerde düşüş bakırın liderliği...
1,77,"İŞKUR personel alımı başladı! İlkokul, ortaoku...",Endüstriyel metallerde düşüş bakırın liderliği...,Sizlere daha iyi bir hizmet sunabilmek için si...,Endüstriyel metallerde düşüş bakırın liderliği...
2,78,Mustafa Cengiz'den Koronavirüs açıklaması!,Fatih Terim: Elim kalbimde teşekkürler,Fatih Terim ve Abdürrahim Albayrak'ın Koronavi...,Koronavirüs teşhisi ile hastanede tedavi altın...
3,79,Adıyaman'da cadde ve sokaklar köpüklendi,Temizlediği silahı ölümüne neden oldu,Adıyaman'da cadde ve sokaklar köpüklendi Adıya...,Temizlediği silahı ölümüne neden oldu Adıyaman...
4,80,Yargılanmayı dışarıda bekleyecek,Corona değil ayrılık krizi,Lefkoşa’da kadın iç çamaşırı çalan zanlı M.S. ...,Suna ERDEN\n\nKendisini terk eden eski sevgili...
5,81,Son dakika: Tarım ve Orman Bakanı Bekir Pakdem...,"Deprem mi oldu, nerede, saat kaçta, kaç şiddet...","Pakdemirli, Twitter'dan gıda işletmelerinin Ko...",Ülkemizin aktif fay hatlarının geçiş güzergahı...
6,82,Londra'da korona salgınında ölen Türk sayısı 1...,Trabzonsporlu futbolcunun acı günü,Londra'da korona salgınında ölen Türk sayısı 1...,"25 Mart 2020 Çarşamba, 10:33\n\nTrabzonsporlu ..."
7,83,"İstanbul Polisi, koronavirüse karşı anonsla uy...",APARTMAN SAKİNLERİNİN VİRÜS TEDBİRİ; MİSAFİR K...,"İstanbul Polisi, koronavirüse karşı anonsla uy...",APARTMAN SAKİNLERİNİN VİRÜS TEDBİRİ; MİSAFİR K...
8,84,AYDIN ADNAN MENDERES ÜNİVERSİTESİ 108 SÖZLEŞME...,Demir ve Züleyha mahkeme salonunda karşı karşı...,Böyle polis olmaz olsun..Olay her ne kadar Man...,Demir ve Züleyha mahkeme salonunda karşı karşı...
9,85,"Güvenlik Kuvvetleri Komutanlığı, Birliklerde A...","MSB: Irak'ın kuzeyinde 2 askerimiz şehit oldu,...","Güvenlik Kuvvetleri Komutanlığı, koronavirüse ...",AA 25 Mart 2020 Çarşamba 22:47 - Güncelleme: 2...


**Corrections**

In [223]:
path = 'C:/Users/stg-sdu/Notebooks/NLP/SemEval-2022/Data/'
eval_data_location = path + "semeval-2022_task8_eval_data_202201.csv"
data_eval_location = pd.read_csv(eval_data_location)

In [224]:
liste_indexes = []
liste_lignes= []
for i in range(len(turc)):
    b = False
    if len(turc.title_1[i]) == 0:
        print('index ',i,' titre 1 : ligne ',turc.ligne[i],data_eval_location.ia_link1[turc.ligne[i]])
        b=True
    if len(turc.text_1[i]) == 0:
        print('index ',i,' texte 1 : ligne ',turc.ligne[i],data_eval_location.ia_link1[turc.ligne[i]])  
        liste_indexes.append(i);liste_lignes.append(i)
        b=True
    if len(turc.title_2[i]) == 0:
        print('index ',i,' titre 2 : ligne ',turc.ligne[i],data_eval_location.ia_link2[turc.ligne[i]])
        b=True
    if len(turc.text_2[i]) == 0:
        print('index ',i,' texte 2 : ligne ',turc.ligne[i],data_eval_location.ia_link2[turc.ligne[i]])
        b = True
    if b == True:
        liste_indexes.append(i);liste_lignes.append(i)

In [216]:
data.iloc[3145]

pair_id                                           1639041939_1637999869
pair_lang                                                         tr_tr
source_url_1                                                           
publish_date_1                                                         
source_url_2                                   https://www.sabah.com.tr
publish_date_2                                 Thu Jun 18 00:00:00 2020
title_1               Güneş Vakfı Başkanı Prof. Dr. Alpaslan Ceylan'...
text_1                Güneş Vakfı Başkanı Prof. Dr. Alpaslan Ceylan,...
meta_description_1                                                     
meta_keywords_1                                                        
title_2               Volkan Bozkır, 178 ülkenin oyuyla BM Genel Kur...
text_2                Birleşmiş Milletler (BM) 75. Genel Kurul Başka...
meta_description_2                                                     
meta_keywords_2                                                 

In [138]:
#data.title_2[3055] = copie_data.title_2[3055]

In [213]:
data.title_1[3145] = "Güneş Vakfı Başkanı Prof. Dr. Alpaslan Ceylan'dan Volkan Bozkır'a tebrik mesajı"

In [214]:
path = (''+
"Güneş Vakfı Başkanı Prof. Dr. Alpaslan Ceylan, Birleşmiş Milletler 75. Genel Kurul Başkanı seçilen Volkan Bozkır'ı tebrik etti."+
"Ceylan açıklamalarında şu ifadeleri kullandı. Birleşmiş Milletler'in 75. Genel Kurul Başkanı seçilen Sayın Volkan Bozkır'ı tebrik ediyorum."+
"2016 yılında Avrupa Birliği Bakanlığı görevinde bulanan Sayın Volkan Bozkır'ı, Güneş Vakfı olarak düzenlemiş olduğumuz 4. Uluslararası Türk Şöleninde misafir etmiştik. Şölende ilkokul öğrenimini gördüğü şehirde bulunmaktan mutluluk duyduğunu belirten Sayın Bozkır, şimdi ise 193 ülkenin temsil edildiği BM Genel Kurulunda 178 ülkenin desteğini alarak BM Genel Kurulunun 75. dönem başkanlığına seçildi. Bu önemli görevinde sayın Bozkır'a başarı dileklerimizi sunuyor ve Türkiye'nin uluslararası arenada göstermiş olduğu başarıları görmekten gurur duyduğumuzu ifade etmek istiyoruz. - ERZURUM")

In [215]:
data.text_1[3145] = path

In [217]:
data.to_csv('eval_data_prep_v1.csv',index=False)

**programme**

In [16]:
# Fonction de calcul du score (produit scalaire) pour résultats de classifaction
def fonction_produit_dotcom(liste_categor, dico_scores1,dico_scores2):
    """"dico scores sont les résultats obtenus pour chaque catégorie des textes 1 et 2"""
    result = 0.0
    for cat in liste_categor:
        result += round(dico_scores1[cat] * dico_scores2[cat],4)
    return result * 100

In [17]:
# transformation des résultats du transformer type1
def transform_text_clf1(liste_dico):
    res = {}
    for dic in liste_dico:
        res[dic['label']] = dic['score']
    return res

In [18]:
# transformation des résultats du transformer type2
def transform_text_clf2(liste_cat,liste_sc):
    res = {}
    for i in range(len(liste_cat)):
        res[liste_cat[i]] = liste_sc[i]
    return res

In [20]:
# Tests
liste_categories = ['world','economy','technology','health','politics','sport','culture']
liste_labels = ['negative','positive']
liste_sentiments = ['ENTAILMENT','NEUTRAL','CONTRADICTION']

In [21]:
# Fonctions de summarization 
def summarization(texte):
    return summarizer1(texte), summarizer2(texte)[0]['summary_text']

In [22]:
dico_classifiers = {'text_clf1': 'score_classif1','text_clf2':'score_classif2','sentiment1':'score_sentiment1',
                    'sentiment2': 'score_sentiment2'}
dico_categories = {'text_clf1': liste_categories,'text_clf2':candidate_labels,'sentiment1':liste_labels,
                    'sentiment2': liste_sentiments}

In [23]:
# Fonctions de classification et sentiment analysis
def classification(texte,clf):
    # assume nms des classifiers et methode de transformation
    if clf == "text_clf1":
        try:
            scores = transform_text_clf1(text_clf1(texte,return_all_scores=True)[0])
        except:
            return 'error'
        else:
            return scores
    elif clf == "text_clf2":                                 
        try:
            classes = text_clf2(texte,dico_categories['text_clf2'])
        except:
            return 'error'
        else:
            return transform_text_clf2(classes['labels'],classes['scores'])                          
    elif clf == "sentiment1":
        try:
            scores = transform_text_clf1(sentiment1(texte,return_all_scores=True)[0])
        except:
            return 'error'
        else:
            return scores
    elif clf == "sentiment2":
        try:
            scores = transform_text_clf1(sentiment2(texte,return_all_scores=True)[0])
        except:
            return 'error'
        else:
            return scores
    else:
        return 'error'

In [24]:
# Prétraitement NLP pour PKE : suppression des mots de moins de 2 lettres non numériques
def supp_moins_2_lettres_stopwords(phrase,stopwd):
    temp = phrase.split(' ')
    res = ''
    for mot in temp:
        if mot not in stopwd and (len(mot)>2 or (len(mot)>0 and mot[0] in ['0','1','2','3','4','5','6','7','8','9'])):
            res += mot + ' '
    return res[:-1]

In [25]:
# Prétraitement NLP pour PKE : suppression des traits d'union(regroupe)/ des apostrophes / ponctuations
def modif(texte,stopmots):
    # modifications simples des textes : ponctuations, petits mots, stopwords (à faire pour entités et pke textes)
    texte=re.sub('\'',' ',texte)   # suppression apostrophe
    texte=re.sub('-','',texte)    # suppression trait union
    regex = re.compile('[%s]' % re.escape(string.punctuation)) # suppression de toutes les ponctuations
    texte=regex.sub(' ',texte)
    texte = supp_moins_2_lettres_stopwords(texte,stopmots)
    return texte

In [26]:
# Ajout des synonymes (existants en orthographe) à la suite de l'analyse pke
def ajout_synonymes(mot, correct_ortho = False):
    # on ajoute les 10 premiers synonymes existants, on vérifie orthographe (optionnel)
    syns = model_gensim.most_similar(mot,topn = 20)
    if correct_ortho == True:
        res = []
        for m in syns:
            if d.check(m[0]):   #  il y a le mot et son pourcentage d'importance
                res.append(m)
        syns = res
    return syns[:10]

In [27]:
# Choix des paramètres de la méthode : A revoir ?
methode1 = {"NOUN", "PROPN", "ADJ","VERB"}
methode2 = {"NOUN", "PROPN", "ADJ"}
nb_mots = {'meth1': 30, 'meth2':50}

In [28]:
# PKE : Analyse des termes principaux dans les textes et titres 
# Problème 
def transformation_pke_results(res1,res2, correct_ortho = False):
    """
    Transformation des resultats de PKE : Pb bigramme peuvent ne pas être ds les 2 textes mais 1 mot seulement
    liste de clés et dictionnaires de valeurs, bigrammes jouera ainsi de maniere coefficientée 
    Exemple : fuite eau:0.05 --> 3 mots au final : fuite, eau, fuite eau : 0.05
    De plus on ajoute les synonymes issus de gensim en les coefficiant et vérifiant que cela """
    
    liste1 = []; liste2 = [] ; dico1 = {}; dico2 = {}
    for elt in res1:
        liste1.append(elt[0])
        dico1[elt[0]] = round(elt[1],3)
        if ' ' in elt[0]:    # bigramme dans ce cas, ajout des 2 mots
            liste = elt[0].split(' ')
            for mot in liste:
                liste1.append(mot)
                dico1[mot] = round(elt[1],3)
                try:
                    synonyms = ajout_synonymes(mot,correct_ortho = correct_ortho)
                except:
                    pass
                else:
                    for syn in synonyms:
                        liste1.append(syn[0])   # Ajout du mot 
                        dico1[syn[0]] = round(elt[1] * syn[1], 3)  # poids considéré
                    
    for elt in res2:
        liste2.append(elt[0])
        dico2[elt[0]] = round(elt[1],3)
        if ' ' in elt[0]:
            liste = elt[0].split(' ')
            for mot in liste:
                liste2.append(mot)
                dico2[mot] = round(elt[1],3)
                try:
                    synonyms = ajout_synonymes(mot,correct_ortho = correct_ortho)
                except:
                    pass
                else:
                    for syn in synonyms:
                        liste2.append(syn[0])   # Ajout du mot 
                        dico2[syn[0]] = round(elt[1] * syn[1], 3)  # poids considéré
    
    # similarites entre les 2 listes issus de pke avec poids
    sim = 0
    for elt in liste1:
        if elt in liste2:
            sim += (dico1[elt] + dico2[elt])/2
    return sim

In [29]:
def entites_communes(nlp,text1,text2):
    """"
    Cette première fonction ne regarde que les entités communes : personnes, dates, groupe, localisations
    Elle sera appliquée aux textes et aux titres et cumulé : si cumul en titre et texte : compte double !"""
    
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    nb_commun_ent = 0; liste_commun_ent = []
    nb_commun_geo = 0; liste_commun_geo = []
    nb_commun_dat = 0; liste_commun_dat = []
    if len(doc1)>0 and len(doc2)>0:
        liste1 = []; dico1 = {}
        for elt in doc1:
            if '-' in elt['entity'] and elt['entity'].split('-')[1] in ['PERSON','PER','LOC','ORG','GPE','NORP','DATE','TIME']:
                liste1.append(elt['word'])
                dico1[elt['word']] = elt['entity'].split('-')[1]
            elif elt['entity'] in ['PERSON','PER','LOC','ORG','GPE','NORP','DATE','TIME']:
                liste1.append(elt['word'])
                dico1[elt['word']] = elt['entity']  
        liste2 = []
        for elt in doc2:
            if '-' in elt['entity'] and elt['entity'].split('-')[1] in ['PERSON','PER','LOC','ORG','GPE','NORP','DATE','TIME']:
                liste2.append(elt['word'])
            elif elt['entity'] in ['LOC','ORG','GPE','NORP','DATE','TIME']:
                liste2.append(elt['word'])
        
        # points communs des listes    
        for elt in liste1:
            if elt in liste2:
                if dico1[elt] == 'LOC':
                    nb_commun_geo += 1
                    liste_commun_geo.append(elt)
                elif dico1[elt] in ['DATE','TIME']:
                    nb_commun_dat += 1
                    liste_commun_dat.append(elt)
                else:
                    nb_commun_ent += 1
                    liste_commun_ent.append(elt)
                    
    return nb_commun_ent, liste_commun_ent,nb_commun_geo, liste_commun_geo,nb_commun_dat, liste_commun_dat

In [30]:
def Creation_features_comparaison(df,langue, test_position = [methode1,methode2]):
    """Création des notes pour classification ensuite"""
    
    resultats = pd.DataFrame(columns = ['summary1_text1','summary2_text1','summary1_text2','summary2_text2',
            'nb_entites_idem','nb_lieux_idem', 'nb_dates_idem','entites_idem','lieux_idem','dates_idem',
            'score_similarite_titres','score_similarite_resume1','score_similarite_resume2','score_classif1','score_classif2',
            'score_sentiment1','score_sentiment2','meth1_similarites','meth2_similarites'])
    
#     # initialisation de la langue stanza
#     stanza.download(langue)
#     # nlp_stanza = spacy_stanza.load_pipeline(langue)
#     nlp_stanza = stanza.Pipeline(langue)   # marche en arabe - changer elt.type au lieu de label
    # en turc : il n'y a pas de NER si stanza ni spacy
    stopmts = stopwds_lg[langue]
#     if langue in dico_spacy.keys():
#         nlp_spacy = dico_spacy[langue]
#     else:
    nlp_spacy = None; nlp_stanza = ner
        
    for i in tqdm(range(len(df))):
        dico_res = {}
        
        # Summary et comparatifs 
        dico_res['summary1_text1'],dico_res['summary2_text1'] = summarization(df.text_1[i])
        dico_res['summary1_text2'],dico_res['summary2_text2'] = summarization(df.text_2[i])
        dico_res['score_similarite_titres'] = score_similarite(df.title_1[i],df.title_2[i])
        dico_res['score_similarite_resume1'] = score_similarite(dico_res['summary1_text1'],dico_res['summary1_text2'])
        dico_res['score_similarite_resume2'] = score_similarite(dico_res['summary2_text1'],dico_res['summary2_text2'])
        
        # analyse de textes classification et de sentiments
        texte1 = df.title_1[i] + ' ' + df.text_1[i]
        texte2 = df.title_2[i] + ' ' + df.text_2[i]
        if len(texte1)>0 and len(texte2)>0:
            for classifier in dico_classifiers.keys():
                scores1 = classification(texte1,classifier)
                scores2 = classification(texte2,classifier)
                if scores1 != 'error' and scores2 != 'error':
                    dico_res[dico_classifiers[classifier]] = fonction_produit_dotcom(dico_categories[classifier], scores1,scores2)
                else:
                    scores1 = classification(df.title_1[i],classifier)
                    scores2 = classification(df.title_2[i],classifier)
                    if scores1 != 'error' and scores2 != 'error':
                        dico_res[dico_classifiers[classifier]] = fonction_produit_dotcom(dico_categories[classifier], scores1,scores2)
                    else:
                        dico_res[dico_classifiers[classifier]] = None
                
        # pré traitement des textes pour entités et PKE
        texte1 = modif(texte1, stopmts)
        texte2 = modif(texte2, stopmts)
        
        # ENTITES COMMUNES : on tient compte des bigrammes Noms qui posent erreurs ex: Joe Biden et Biden 
        # Ici, on considère mieux le CUMUl titres et Textes avec une pondération double pour le titre 
        # Il faut aussi enlever les petits mots donc pré-traitement en texte
        
        nb_ent1,list_ent1,nb_geo1,list_geo1,nb_dat1,list_dat1 = entites_communes(nlp_stanza,df.title_1[i],df.title_2[i])
        nb_ent2,list_ent2,nb_geo2,list_geo2,nb_dat2,list_dat2 = entites_communes(nlp_stanza,df.text_1[i],df.text_2[i])
        if nlp_spacy != None:
            nb_ent3,list_ent3,nb_geo3,list_geo3,nb_dat3,list_dat3 = entites_communes(nlp_spacy,df.title_1[i],df.title_2[i])
            nb_ent4,list_ent4,nb_geo4,list_geo4,nb_dat4,list_dat4 = entites_communes(nlp_spacy,df.text_1[i],df.text_2[i])
        else:
            nb_ent3,list_ent3,nb_geo3,list_geo3,nb_dat3,list_dat3 = (0,[],0,[],0,[])
            nb_ent4,list_ent4,nb_geo4,list_geo4,nb_dat4,list_dat4 = (0,[],0,[],0,[])
        dico_res['nb_entites_idem'] = nb_ent1 * 2 + nb_ent2 + nb_ent3 * 2 + nb_ent4
        dico_res['nb_lieux_idem'] = nb_geo1  * 2 + nb_geo2 + nb_geo3  * 2 + nb_geo4
        dico_res['nb_dates_idem'] = nb_dat1 * 2 + nb_dat2 + nb_dat3 * 2 + nb_dat4
        # fusion des listes en supprimant les doublons
        dico_res['entites_idem'] = list(set(list_ent1+list_ent2+ list_ent3+list_ent4))
        dico_res['lieux_idem'] = list(set(list_geo1+list_geo2+list_geo3+list_geo4))
        dico_res['dates_idem'] = list(set(list_dat1+list_dat2+list_dat3+list_dat4))
        
        for j,meth in enumerate(test_position):
            nom ='meth'+str(j+1)
            nb_mots_meth = nb_mots[nom]
            if len(texte1)>0 and len(texte2)>0:
                extractor = pke.unsupervised.TopicRank()
                extractor.load_document(input=texte1,language=langue,normalization="stemming")
                extractor.candidate_selection(pos=meth)
                extractor.candidate_weighting()
                keyphrases3 = extractor.get_n_best(n=nb_mots_meth)
                extractor = pke.unsupervised.TopicRank()
                extractor.load_document(input=texte2,language=langue,normalization="stemming")
                extractor.candidate_selection(pos=meth)
                extractor.candidate_weighting()
                keyphrases4 = extractor.get_n_best(n=nb_mots_meth)
                dico_res[nom+'_similarites'] = round(100*transformation_pke_results(keyphrases3,keyphrases4),1)
            else:
                dico_res[nom+'_similarites'] = 'Error'
        
        resultats.loc[len(resultats)] = dico_res
        
    newdf = pd.concat([df,resultats],axis=1)
    return newdf

In [226]:
# Attention probleme au 16e index non identifié ...
similarites = Creation_features_comparaison(turc.reset_index(drop=True),'tr')

  0%|          | 0/275 [00:00<?, ?it/s]

Your max_length is set to 20, but you input_length is only 18. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


In [227]:
similarites.to_csv('eval_tr_notes.csv')

In [228]:
similarites

,ligne,title_1,title_2,text_1,text_2,summary1_text1,summary2_text1,summary1_text2,summary2_text2,nb_entites_idem,...,dates_idem,score_similarite_titres,score_similarite_resume1,score_similarite_resume2,score_classif1,score_classif2,score_sentiment1,score_sentiment2,meth1_similarites,meth2_similarites
0,76,ŞOK 25-31 Mart Aktüel ürünler kataloğu 2020! Ş...,Endüstriyel metallerde düşüş bakırın liderliği...,Sizlere daha iyi bir hizmet sunabilmek için si...,Endüstriyel metallerde düşüş bakırın liderliği...,sizlere daha iyi bir hizmet sunabilmek için si...,"Geçtiğimiz günlerde , 'Çerez Politikasını' ve 'G",Endüstriyel metalde gösterge kabul edilen bakı...,Endüstriyel metallerde düşüş bakırın liderliği...,0,...,[],8.73,5.96,13.42,99.49,9.79,23.63,59.69,0,0
1,77,"İŞKUR personel alımı başladı! İlkokul, ortaoku...",Endüstriyel metallerde düşüş bakırın liderliği...,Sizlere daha iyi bir hizmet sunabilmek için si...,Endüstriyel metallerde düşüş bakırın liderliği...,sizlere daha iyi bir hizmet sunabilmek için si...,"Geçtiğimiz günlerde , 'Çerez Politikasını' ve 'G",Endüstriyel metalde gösterge kabul edilen bakı...,Endüstriyel metallerde düşüş bakırın liderliği...,0,...,[],5.06,5.96,13.42,54.41,9.37,25.26,22.80,0,0
2,78,Mustafa Cengiz'den Koronavirüs açıklaması!,Fatih Terim: Elim kalbimde teşekkürler,Fatih Terim ve Abdürrahim Albayrak'ın Koronavi...,Koronavirüs teşhisi ile hastanede tedavi altın...,"Galatasaray Başkanı Mustafa Cengiz, Koronavirü...","Galatasaray Başkanı Mustafa Cengiz , Galatasar...","Galatasaray Teknik Direktörü Fatih Terim, koro...","Galatasaray Teknik Direktörü Fatih Terim'in , ...",12,...,[],16.38,34.78,51.54,99.57,9.98,14.47,49.92,0,0
3,79,Adıyaman'da cadde ve sokaklar köpüklendi,Temizlediği silahı ölümüne neden oldu,Adıyaman'da cadde ve sokaklar köpüklendi Adıya...,Temizlediği silahı ölümüne neden oldu Adıyaman...,Adıyaman'da cadde ve sokaklar köpüklü su ile e...,Adıyaman'da cadde ve sokaklar köpüklendi Adıya...,"Adıyaman'da, silahını temizlediği esnada kazar...","ADIYAMAN'da , silahını temizlediği esnada kaza...",0,...,[],3.60,4.40,7.35,82.01,27.75,51.80,97.78,0,0
4,80,Yargılanmayı dışarıda bekleyecek,Corona değil ayrılık krizi,Lefkoşa’da kadın iç çamaşırı çalan zanlı M.S. ...,Suna ERDEN\n\nKendisini terk eden eski sevgili...,Lefkoşa ’ da kadın iç çamaşırı çalan zanlı M. ...,İSTANBUL Lefkoşa'da kadın iç çamaşırı çalan za...,KKTC Kuzey Kıbrıs Türk Cumhuriyeti ( KKTC ) Es...,Suna ERDEN Kendisini terk eden eski sevgilisin...,3,...,[],3.70,30.70,20.78,51.06,9.95,48.44,98.89,79.9,266.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,3148,YKS müfredatı belli oldu! 12. sınıf ilk dönem ...,YKS’de son viraj,12. sınıf müfredatı nedir? YKS'de 12. sınıf mü...,Binlerce adayın üniversiteli olmak için ter dö...,YKS'de 12. sınıf müfredatının nerelerinden sor...,MEB'in 12. sınıf müfredatı ne zaman soruları b...,Bilfen Liseleri Rehberlik Bölüm Başkanı Sebaha...,Yükseköğretim Kurumları Sınavı ( YKS ) 27-28 H...,30,...,[],16.53,62.71,21.57,22.19,9.98,28.94,14.81,328.2,200.4
271,3149,Hafta sonu sokağa çıkma yasağı yeniden geliyor,İçişleri Bakanı Süleyman Soylu'nun olay yarata...,Kabine toplantısının sona ermesinin ardından E...,"İçişleri Bakanı Süleyman Soylu, cuma gecesi al...","Cumhurbaşkanı Recep Tayyip Erdoğan, Kabine'dek...","Cumhurbaşkanı Recep Tayyip Erdoğan , İçişleri ...","İçişleri Bakanı Süleyman Soylu, cuma gecesi al...","İçişleri Bakanı Süleyman Soylu, cuma gecesi al...",4,...,[],-2.62,43.42,47.37,97.55,10.02,82.60,94.64,8.2,74.2
272,3150,Oltan TZK’da gol krallığına yürüyor,Erzurumspor'da Erkan Sözeri dönemi sona erdi!,BB Erzurumspor’un forvet oyuncusu Oltan Karaku...,TFF 1. Lig ekiplerinden BB Erzurumspor sahasın...,BB Erzurumspor ’ un forvet oyuncusu Oltan Kara...,BB Erzurumspor'un forvet oyuncusu Oltan Karaku...,"TFF 1. Lig ekiplerinden BB Erzurumspor, Teknik...",TFF 1. Lig ekiplerinden BB Erzurumspor sahasın...,28,...,[],9.73,25.87,43.13,99.83,10.00,45.33,6.79,134,331
273,3151,Wa